# *******************************************************
# IGTI - Bootcamp Analista de Machine Learning
# Módulo 3 - Seleção de Modelos de ML
# TRABALHO PRATICO - GISELLE
# *******************************************************

In [7]:
from sklearn import datasets
from sklearn.decomposition import PCA

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split,KFold,cross_val_score, cross_val_predict 
from sklearn.metrics import mean_absolute_error #utilizada para o calculo do MAE
from sklearn.metrics import mean_squared_error #utilizada para o calculo do MSE
from sklearn import metrics  #utilizada para as métricas de comparação entre os métodos
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV  #busca exaustiva em grid
from sklearn.model_selection import cross_val_score

**Conhecendo o banco de dados**

In [8]:
iris = datasets.load_iris()  #dataset IRIS

In [9]:
df_iris = pd.DataFrame(data= np.c_[iris['data'], iris['target']],   # variável target = saída (especie que a planta pertence)
                     columns= iris['feature_names'] + ['target'])

In [10]:
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [11]:
#Classe a ser usada nos modelos de classificação
df_iris['target'].value_counts()

2.0    50
1.0    50
0.0    50
Name: target, dtype: int64

In [12]:
df_iris.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


**Algoritmos de Classificação**

In [13]:
X = iris.data[:, :2] # pegamos as primeiras duas features aqui (apenas sepal lenght e sepal with). Pegue as outras se quiser.
y = iris.target      #classificação das flores (0 1 2)

In [14]:
y[0] # classe do primeiro elemento
X[:1, :] # primeiro elemento a classificar

array([[5.1, 3.5]])

**Buscando os Hiperparâmetros para o Random Forest**

In [15]:
# RFC - Random Forest Classifier
rfc = RandomForestClassifier()

parameters = {'min_samples_leaf':(5,20)}

rfc_iris = GridSearchCV(rfc, parameters)  #rfc -> já instanciado anteriormente neste código
rfc_iris.fit( X, y)  #com todos os dados, pois o próprio método de validação faz as devidas separações
print("Melhor valor para 'min_samples_leaf': {0}".format( rfc_iris.best_params_['min_samples_leaf']))

Melhor valor para 'min_samples_leaf': 5


**Ajustando o modelo RF**

In [16]:
X_RF_train, X_RF_test, y_RF_train, y_RF_test = train_test_split( X, y, test_size = 0.30, random_state = 42)

In [17]:
rfc = RandomForestClassifier(min_samples_leaf=5)  #usando o valor de hiperparâmetro definido no GridSearch
rfc.fit( X_RF_train, y_RF_train)     #modelo treinado
y_RF_pred = rfc.predict( X_RF_test)  #teste do modelo

In [18]:
rfc_acc = round( accuracy_score( y_RF_test, y_RF_pred), 6) # round é para arredondar
rfc_recall = round( recall_score( y_RF_test, y_RF_pred, average = 'weighted'), 6) # round é para arredondar
rfc_precision = round( precision_score( y_RF_test, y_RF_pred, average = 'weighted'), 6) # round é para arredondar

print("Acurácia:",rfc_acc," Recall:", rfc_recall," Precisão:", rfc_precision)

Acurácia: 0.777778  Recall: 0.777778  Precisão: 0.779394


**Buscando os Hiperparâmetros para o KNN**

In [19]:
# KNN
knn = KNeighborsClassifier()

parameters = {'n_neighbors':(3,10)}
knn_iris = GridSearchCV( knn, parameters)
knn_iris.fit( X, y)

print("Melhor valor para 'n_neighbors': {0}".format( knn_iris.best_params_['n_neighbors']))

Melhor valor para 'n_neighbors': 10


In [29]:
knn_iris.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [31]:
knn_iris.best_score_  # resultado da validação cruzada (média das acurácias)

0.7666666666666667

**Ajustando o modelo KNN**

In [20]:
X_KNN_train, X_KNN_test, y_KNN_train, y_KNN_test = train_test_split( X, y, test_size = 0.30, random_state = 42)

In [21]:
knn = KNeighborsClassifier( n_neighbors = 10)
knn.fit( X_KNN_train, y_KNN_train)
y_KNN_pred = knn.predict( X_KNN_test)

In [22]:
rfc_acc = round( accuracy_score( y_KNN_test, y_KNN_pred), 6) # round é para arredondar
rfc_recall = round( recall_score( y_KNN_test, y_KNN_pred, average = 'weighted'), 6) # round é para arredondar
rfc_precision = round( precision_score( y_KNN_test, y_KNN_pred, average = 'weighted'), 6) # round é para arredondar

print("Acurácia:",rfc_acc," Recall:", rfc_recall," Precisão:", rfc_precision)

Acurácia: 0.755556  Recall: 0.755556  Precisão: 0.75582


**Validação Cruzada**

In [23]:
# Validação Cruzada

cv_knn = cross_val_score( knn, X, y)
cv_rfc = cross_val_score( rfc, X, y)

print( "\nValidação Cruzada KNN: {}".format( cv_knn))
print( "\nValidação Cruzada RFC: {}".format( cv_rfc))


Validação Cruzada KNN: [0.7        0.76666667 0.8        0.83333333 0.73333333]

Validação Cruzada RFC: [0.7        0.83333333 0.7        0.8        0.86666667]


In [24]:
# Verifica médias
# RFC

sum_cv_rfc = 0

for cv_score in cv_rfc:
  sum_cv_rfc = sum_cv_rfc + cv_score

In [25]:
# Verifica médias
# KNN

sum_cv_knn = 0

for cv_score in cv_knn:
  sum_cv_knn = sum_cv_knn + cv_score

In [26]:
print("\nResultado RFC: {0}".format( sum_cv_rfc/5))
print("\nResultado KNN: {0}".format( sum_cv_knn/5))


Resultado RFC: 0.78

Resultado KNN: 0.7666666666666667


In [34]:
# ou poderia ser feito da seguinte forma usando o GridSearch:
print("\nResultado RFC: {0}".format(rfc_iris.best_score_))  # resultado da validação cruzada (média das acurácias)
print("\nResultado KNN: {0}".format(knn_iris.best_score_))  # resultado da validação cruzada (média das acurácias)


Resultado RFC: 0.7866666666666666

Resultado KNN: 0.7666666666666667
